# Worksheet 10.2 Finite Difference Method for Boundary Value Problems
## ENGR 240: Engineering Computations

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%2010/Worksheet%2010-2%20Finite%20Difference%20BVPs.ipynb)

## Introduction: Municipal Groundwater Well Field Analysis

Modern cities depend on groundwater wells for municipal water supply. Understanding the hydraulic head distribution in an aquifer is critical for:
- **Sustainable pumping rates** - preventing aquifer depletion
- **Well interference analysis** - optimizing well spacing
- **Environmental protection** - maintaining base flow to rivers
- **Water quality management** - predicting contaminant transport

### Physical System

**Aquifer Configuration:**
- **Length**: 2000 m (from river to impermeable bedrock)
- **Material**: Sand and gravel aquifer
- **Hydraulic conductivity**: K = 1.5 × 10⁻⁴ m/s
- **Thickness**: 25 m (confined aquifer)
- **River boundary**: Constant head at 120 m elevation
- **Bedrock boundary**: Impermeable with regional gradient

**Well System:**
- **Well 1**: Located 400 m from river, pumping 0.05 m³/s
- **Well 2**: Located 1200 m from river, pumping 0.03 m³/s

### Mathematical Model

The steady-state groundwater flow equation with pumping wells:
$$\frac{d^2h}{dx^2} = \frac{Q(x)}{T}$$

**Boundary Conditions:**
- **River (x = 0)**: Fixed head $h(0) = 120$ m
- **Bedrock (x = L)**: Prescribed gradient $\frac{dh}{dx}\bigg|_{x=L} = -0.001$

**Learning Objectives:**
- Convert BVP to linear system using finite differences
- Handle non-zero boundary conditions in matrix formulation
- Understand mesh refinement effects
- Apply numerical methods to water resources engineering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
print("Libraries imported successfully!")

## System Parameters

In [ ]:
# Physical parameters
L = 2000.0          # Aquifer length (m)
K = 1.5e-4          # Hydraulic conductivity (m/s)
b = 25.0            # Aquifer thickness (m)
T = K * b           # Transmissivity (m²/s)
h_river = 120.0     # River head (m)
grad_bedrock = -0.001  # Regional gradient at bedrock boundary

# Well locations and pumping rates
well_1_location = 400.0   # m from river
well_1_rate = -0.05       # m³/s (negative for pumping)
well_2_location = 1200.0  # m from river  
well_2_rate = -0.03       # m³/s (negative for pumping)

# Display parameters
print("=== Municipal Groundwater Well Field Parameters ===")
print(f"Aquifer length: {L:.0f} m")
print(f"Hydraulic conductivity: {K:.1e} m/s")
print(f"Transmissivity: {T:.1e} m²/s")
print(f"River head (boundary): {h_river:.0f} m")
print(f"Bedrock gradient: {grad_bedrock:.3f}")
print(f"Well 1: {well_1_rate:.3f} m³/s at {well_1_location:.0f} m")
print(f"Well 2: {well_2_rate:.3f} m³/s at {well_2_location:.0f} m")

## Task 1: Understanding Boundary Value Problems vs Initial Value Problems

Before diving into finite differences, let's understand why this is a **boundary value problem (BVP)** rather than an **initial value problem (IVP)**.

### Key Differences:

| Aspect | Initial Value Problem | Boundary Value Problem |
|--------|----------------------|------------------------|
| **Conditions** | All conditions at one point (t=0) | Conditions at multiple boundaries |
| **Solution method** | March forward in time/space | Solve entire domain simultaneously |
| **Physical meaning** | Time evolution | Steady-state spatial distribution |
| **Examples** | Population growth, projectile motion | Beam deflection, heat conduction |

### Our Groundwater Problem:
- We know head at the **river** (x=0): h(0) = 120 m
- We know gradient at **bedrock** (x=L): dh/dx = -0.001
- We need to find h(x) everywhere in between
- This requires solving the **entire domain simultaneously**

In [ ]:
# Illustrate the BVP setup
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# BVP schematic
x_schematic = np.linspace(0, L, 100)
ax1.plot([0, L], [h_river, h_river-2], 'b-', linewidth=3, alpha=0.3, label='Unknown solution h(x)')
ax1.plot(0, h_river, 'ro', markersize=12, label=f'River: h(0) = {h_river} m')
ax1.arrow(L-100, 115, 80, -1, head_width=20, head_length=30, fc='orange', ec='orange')
ax1.text(L-200, 112, f'Bedrock: dh/dx = {grad_bedrock}', fontsize=12, ha='center')
ax1.scatter([well_1_location, well_2_location], [118, 116], c='red', s=100, marker='v', label='Pumping wells')
ax1.set_xlabel('Distance from river (m)')
ax1.set_ylabel('Hydraulic head (m)')
ax1.set_title('Boundary Value Problem Setup')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Why we can't use IVP methods
x_ivp = np.linspace(0, 500, 100)
h_guess1 = h_river + x_ivp * (-0.01)  # Too steep slope
h_guess2 = h_river + x_ivp * (0.005)  # Wrong direction
h_guess3 = h_river + x_ivp * (-0.002) # Better guess

ax2.plot(x_ivp, h_guess1, 'r--', label='Guess 1: slope = -0.01')
ax2.plot(x_ivp, h_guess2, 'g--', label='Guess 2: slope = +0.005')
ax2.plot(x_ivp, h_guess3, 'b--', label='Guess 3: slope = -0.002')
ax2.plot(0, h_river, 'ro', markersize=12, label=f'Known: h(0) = {h_river} m')
ax2.set_xlabel('Distance from river (m)')
ax2.set_ylabel('Hydraulic head (m)')
ax2.set_title('Why IVP Methods Fail Here')
ax2.text(250, 122, 'Which slope is correct?\nWe need ALL boundary\nconditions simultaneously!', 
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: BVPs require solving the entire spatial domain simultaneously!")
print("The finite difference method converts the BVP into a system of linear equations.")

## Task 2: Hand Formulation - 6 Node System

Now let's work through the **core exercise**: manually setting up the coefficient matrix for a 6-node system.

### 6-Node Discretization Setup:

| Parameter | Value |
|-----------|-------|
| Domain length (L) | 2000 m |
| Number of nodes | 6 |
| Grid spacing (Δx) | L/(n-1) = 2000/5 = 400 m |
| Transmissivity (T) | 3.75×10⁻³ m²/s |
| Well 1 location | Node 2 (x = 400 m) |
| Well 2 location | Node 4 (x = 1200 m) |

In [ ]:
# 6-node system setup
n_nodes_6 = 6
dx_6 = L / (n_nodes_6 - 1)
x_6 = np.linspace(0, L, n_nodes_6)

print("=== 6-Node System Setup ===")
print(f"Grid spacing: Δx = {dx_6:.0f} m")
print(f"Transmissivity: T = {T:.3e} m²/s")
print("\nNode locations:")
for i, x_val in enumerate(x_6):
    print(f"Node {i+1}: x = {x_val:.0f} m")

# Well pumping rates at each node
Q_6 = np.zeros(n_nodes_6)
Q_6[1] = well_1_rate  # Node 2 (index 1)
Q_6[3] = well_2_rate  # Node 4 (index 3)

print("\nPumping rates:")
for i, q in enumerate(Q_6):
    if q != 0:
        print(f"Node {i+1}: Q = {q:.3f} m³/s")

# Manual matrix assembly exercise
print("\n=== Manual Matrix Assembly Exercise ===")
print(f"Given: Δx = {dx_6:.0f} m, T = {T:.3e} m²/s")
print(f"Formula: RHS = Q × (Δx)² / T")

# Calculate RHS values for each node
rhs_factor = dx_6**2 / T
print(f"\nRHS factor = (Δx)²/T = {rhs_factor:.3e}")
print("\nInterior node equations:")
print(f"Node 2: h₁ - 2h₂ + h₃ = {Q_6[1]:.3f} × {rhs_factor:.3e} = {Q_6[1] * rhs_factor:.3e}")
print(f"Node 3: h₂ - 2h₃ + h₄ = {Q_6[2]:.3f} × {rhs_factor:.3e} = {Q_6[2] * rhs_factor:.3e}")
print(f"Node 4: h₃ - 2h₄ + h₅ = {Q_6[3]:.3f} × {rhs_factor:.3e} = {Q_6[3] * rhs_factor:.3e}")
print(f"Node 5: h₄ - 2h₅ + h₆ = {Q_6[4]:.3f} × {rhs_factor:.3e} = {Q_6[4] * rhs_factor:.3e}")
print("\nBoundary conditions:")
print(f"Node 1: h₁ = {h_river:.0f} m (substitute into equations above)")
print(f"Node 6: dh/dx = {grad_bedrock:.3f} → h₆ = h₅ + {grad_bedrock * dx_6:.1f}")

In [ ]:
def solve_groundwater_6_nodes_manual():
    """
    Solve the 6-node groundwater system using hand-calculated matrix.
    Students fill in the coefficient matrix A and RHS vector b.
    """
    
    # TODO: Fill in the 5×5 coefficient matrix A
    # Rows represent equations for nodes 2, 3, 4, 5, and the boundary condition
    # Columns represent unknowns h₂, h₃, h₄, h₅, h₆
    
    A = np.array([
        # Node 2: h₁ - 2h₂ + h₃ = RHS₂ → -2h₂ + h₃ = RHS₂ - h₁
        [-2,  1,  0,  0,  0],  # TODO: Verify these coefficients
        
        # Node 3: h₂ - 2h₃ + h₄ = RHS₃
        [ 1, -2,  1,  0,  0],  # TODO: Fill in correct values
        
        # Node 4: h₃ - 2h₄ + h₅ = RHS₄
        [ 0,  1, -2,  1,  0],  # TODO: Fill in correct values
        
        # Node 5: h₄ - 2h₅ + h₆ = RHS₅
        [ 0,  0,  1, -2,  1],  # TODO: Fill in correct values
        
        # Boundary condition: h₆ - h₅ = -0.4
        [ 0,  0,  0, -1,  1]   # TODO: Fill in correct values
    ], dtype=float)
    
    # TODO: Fill in the right-hand side vector b
    rhs_factor = dx_6**2 / T
    
    b = np.array([
        # Node 2 equation: RHS₂ - h₁ (because h₁ = 120 is moved to RHS)
        Q_6[1] * rhs_factor - h_river,  # TODO: Calculate this value
        
        # Node 3 equation: RHS₃
        Q_6[2] * rhs_factor,            # TODO: Calculate this value
        
        # Node 4 equation: RHS₄  
        Q_6[3] * rhs_factor,            # TODO: Calculate this value
        
        # Node 5 equation: RHS₅
        Q_6[4] * rhs_factor,            # TODO: Calculate this value
        
        # Boundary condition: h₆ - h₅ = grad_bedrock * dx
        grad_bedrock * dx_6             # TODO: Calculate this value
    ])
    
    print("Coefficient matrix A:")
    print(A)
    print("\nRight-hand side vector b:")
    print(b)
    
    # Solve the linear system
    h_unknowns = np.linalg.solve(A, b)
    
    # Reconstruct complete solution
    h_complete = np.zeros(n_nodes_6)
    h_complete[0] = h_river        # Node 1 (boundary condition)
    h_complete[1:] = h_unknowns    # Nodes 2-6 (solved values)
    
    return h_complete, A, b

# Test the manual solution
print("Testing your manual matrix assembly:")
try:
    h_manual, A_manual, b_manual = solve_groundwater_6_nodes_manual()
    print("\n✅ Matrix solution successful!")
    print("\nSolution vector (hydraulic heads):")
    for i, h in enumerate(h_manual):
        print(f"Node {i+1} (x={x_6[i]:.0f}m): h = {h:.2f} m")
        
except np.linalg.LinAlgError:
    print("❌ Matrix is singular! Check your coefficient matrix.")
except Exception as e:
    print(f"❌ Error: {e}")

## Task 3: Mesh Refinement Study

Let's study how the solution converges as we increase the number of nodes.

In [ ]:
def solve_groundwater_bvp(n_nodes, Q_wells, well_locations):
    """
    Solve groundwater BVP using finite difference method.
    """
    # Grid setup
    dx = L / (n_nodes - 1)
    x = np.linspace(0, L, n_nodes)
    
    # Initialize pumping rate array
    Q = np.zeros(n_nodes)
    
    # Assign pumping rates to nearest nodes
    for q_well, x_well in zip(Q_wells, well_locations):
        node_idx = np.argmin(np.abs(x - x_well))
        Q[node_idx] += q_well
    
    # Build coefficient matrix
    n_unknowns = n_nodes - 1
    A = np.zeros((n_unknowns, n_unknowns))
    b = np.zeros(n_unknowns)
    
    # Interior nodes
    for i in range(1, n_nodes-1):
        row = i - 1
        
        if i == 1:  # First interior node
            A[row, 0] = -2
            A[row, 1] = 1
            b[row] = Q[i] * dx**2 / T - h_river
        else:
            A[row, i-2] = 1
            A[row, i-1] = -2
            A[row, i] = 1
            b[row] = Q[i] * dx**2 / T
    
    # Right boundary condition
    row = n_unknowns - 1
    A[row, -2] = -1
    A[row, -1] = 1
    b[row] = grad_bedrock * dx
    
    # Solve
    h_interior = np.linalg.solve(A, b)
    
    # Reconstruct complete solution
    h = np.zeros(n_nodes)
    h[0] = h_river
    h[1:] = h_interior
    
    return x, h

# Mesh refinement study
node_counts = [6, 11, 21, 41]
solutions = {}

print("=== Mesh Refinement Study ===")
for n in node_counts:
    x, h = solve_groundwater_bvp(n, [well_1_rate, well_2_rate], 
                                [well_1_location, well_2_location])
    solutions[n] = {'x': x, 'h': h}
    
    min_idx = np.argmin(h)
    min_head = h[min_idx]
    min_location = x[min_idx]
    
    print(f"n={n:2d} nodes: Δx={L/(n-1):6.1f}m, min head = {min_head:.3f}m at x={min_location:.0f}m")

# Plot comparison
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

colors = ['red', 'blue', 'green', 'orange']
for i, n in enumerate(node_counts):
    sol = solutions[n]
    ax.plot(sol['x'], sol['h'], 'o-', color=colors[i], 
             label=f'n={n} (Δx={L/(n-1):.0f}m)', markersize=4, linewidth=2)

ax.set_xlabel('Distance from river (m)')
ax.set_ylabel('Hydraulic head (m)')
ax.set_title('Mesh Refinement Study')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nAs mesh gets finer, solution converges to the true answer!")

## Discussion Questions

1. **How do the boundary conditions affect the solution?**
   - What happens if we change the river level?
   - How does the gradient boundary condition influence flow patterns?

2. **Why can't we solve this problem analytically?**
   - Multiple point sources (wells)
   - Mixed boundary conditions
   - Complex domain geometry

3. **How does mesh refinement affect the solution?**
   - When is the solution "converged"?
   - What are the computational trade-offs?

4. **What engineering insights can we extract?**
   - Well interference effects
   - Sustainable pumping rates
   - Flow budget analysis

## Key Takeaways

- **Finite difference method** converts BVPs to linear algebra problems
- **Non-zero boundary conditions** modify the coefficient matrix and RHS vector
- **Mesh refinement** is essential for accurate solutions
- **Engineering interpretation** transforms numbers into actionable insights

The finite difference method is a powerful tool for solving complex engineering problems where analytical solutions are impossible!